<a href="https://colab.research.google.com/github/larasauser/2024_MLEES/blob/main/Project/project_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**# Machine learning Project - Sauser Lara**
## *Filling gaps in NDVI images using CNN and Diffusion models*

[Ajouter une image d'illustration]

##Main steps :


1.   Gather dataset
2.   Split the dataset
3.   Create the CNN alorithm
4.   Create the Dissolution model
5.   Evaluate them (RMSE ?)


##Questions
1.   Should I keep infos of date when cutting
my images ?



### 1. Loading the data and separate them into smaller images (32x32 px)
--- This should be done only once per image set because it takes time ---

--- New image set should be used after this step ---



In [1]:
#Dataset stored on Google Drive
from google.colab import drive
drive.mount('/content/drive')


import os
folder_path = '/content/drive/My Drive/NDVI_Images_Landsat8_10/'

Mounted at /content/drive


To have more images and to reduce computational cost, we decide to devide them into sub-images of 25x25 pixels.

In [ ]:
#pip install Pillow

In [ ]:
#img=Image.open('/content/drive/My Drive/NDVI_Images_Landsat8_10/NDVI_Val_Herens_2023-11-08.tif')
#img_width, img_height = img.size
#print(img_width)
#print(img_height)

Ajouter de l'overlap ??

In [ ]:
from PIL import Image
import os
def split_image(image_path, output_folder, sub_image_size=(32, 32)):
    # Open image
    img = Image.open(image_path)
    img_width, img_height = img.size

    # Image count
    count = 0

    # Division
    for i in range(0, img_width, sub_image_size[0]):
        for j in range(0, img_height, sub_image_size[1]):
            # Cutting box
            box = (i, j, i + sub_image_size[0], j + sub_image_size[1])
            # Cutting
            sub_image = img.crop(box)
            # Verification (size)
            if sub_image.size[0] > 0 and sub_image.size[1] > 0:
                # Saving
                sub_image.save(os.path.join(output_folder, f"{os.path.splitext(os.path.basename(image_path))[0]}_sub_image_{count}.tif"))
                count += 1
    print(count)

def process_all_images(input_folder, output_folder):
    # List all tif file in input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.tif'):
            image_path = os.path.join(input_folder, filename)
            print(f"Image in treatment : {filename}")
            split_image(image_path, output_folder)


In [ ]:
#Split our images
output_folder = '/content/drive/My Drive/NDVI_Images_Landsat8_10/split'
process_all_images(folder_path, output_folder)

Image in treatment : NDVI_Val_Herens_2013-04-18.tif
600
Image in treatment : NDVI_Val_Herens_2013-11-12.tif
600
Image in treatment : NDVI_Val_Herens_2013-12-30.tif
600
Image in treatment : NDVI_Val_Herens_2013-11-28.tif
600
Image in treatment : NDVI_Val_Herens_2014-03-20.tif
600
Image in treatment : NDVI_Val_Herens_2014-06-08.tif
600
Image in treatment : NDVI_Val_Herens_2014-10-30.tif
600
Image in treatment : NDVI_Val_Herens_2015-02-19.tif
600
Image in treatment : NDVI_Val_Herens_2015-04-24.tif
600
Image in treatment : NDVI_Val_Herens_2015-04-08.tif
600
Image in treatment : NDVI_Val_Herens_2015-08-30.tif
600
Image in treatment : NDVI_Val_Herens_2015-05-10.tif
600
Image in treatment : NDVI_Val_Herens_2015-12-20.tif
600
Image in treatment : NDVI_Val_Herens_2016-10-03.tif
600
Image in treatment : NDVI_Val_Herens_2016-04-10.tif
600
Image in treatment : NDVI_Val_Herens_2016-12-06.tif
600
Image in treatment : NDVI_Val_Herens_2017-03-28.tif
600
Image in treatment : NDVI_Val_Herens_2017-01-23.

Images are now splitted. We now have 13'200 images to use for our model.

### 2. Convolution Neural Network (CNN)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### 2.0 Data Augmentation ???

#### 2.1 Split the dataset into train, test and validation set

In [4]:
import os
import random
import shutil

In [5]:
# Define the source directory and the new directories for train, validation, and test sets
source_dir = '/content/drive/My Drive/NDVI_Images_Landsat8_10/split'
train_dir = '/content/drive/My Drive/NDVI_Images_Landsat8_10/train'
val_dir = '/content/drive/My Drive/NDVI_Images_Landsat8_10/validation'
test_dir = '/content/drive/My Drive/NDVI_Images_Landsat8_10/test'

In [6]:
# Create the new directories if they don't exist
for dir in [train_dir, val_dir, test_dir]:
    if not os.path.exists(dir):
        os.makedirs(dir)

In [7]:
# Get all image files from the source directory
all_images = [f for f in os.listdir(source_dir) if f.endswith(('.tif', '.tiff'))]

In [8]:
# Shuffle the list of images
random.shuffle(all_images)

In [9]:
# Calculate the number of images for each set
total_images = len(all_images)
train_split = int(0.7 * total_images)
val_split = int(0.2 * total_images)

In [10]:
# Split the images into train, validation, and test sets
train_images = all_images[:train_split]
val_images = all_images[train_split:train_split+val_split]
test_images = all_images[train_split+val_split:]

In [11]:
# Function to copy images to their respective directories
def copy_images(image_list, destination):
    for image in image_list:
        src = os.path.join(source_dir, image)
        dst = os.path.join(destination, image)
        shutil.copy(src, dst)  # Use shutil.copy instead of shutil.move


In [12]:
# Copy the images to their respective directories
copy_images(train_images, train_dir)
copy_images(val_images, val_dir)
copy_images(test_images, test_dir)

In [13]:
print(f"Total images: {total_images}")
print(f"Training images: {len(train_images)}")
print(f"Validation images: {len(val_images)}")
print(f"Test images: {len(test_images)}")

Total images: 1928
Training images: 1349
Validation images: 385
Test images: 194


#### 2.2 Create fake holes in images for training

In [ ]:
import os
import numpy as np
import cv2
from tqdm import tqdm